# Implementing and Testing the KNN Algorithm on the Iris Dataset

In this notebook, we implement the K-Nearest Neighbors (KNN) algorithm from scratch using only the `NumPy` library. Then, we will test, optimize, and evaluate the implemented model on the famous Iris dataset.

Workflow Steps:
1.  **Section 1:** Implement the `MyKNNClassifier` class in a `knn.py` file.
2.  **Section 2:** Load and prepare the Iris dataset (including scaling and splitting the data).
3.  **Section 3:** Use `GridSearchCV` and `KFold` (Cross-Validation) to find the best hyperparameters (K, distance metric, and weighting mode).
4.  **Section 4:** Final `classification_report` (including Accuracy, Precision, Recall) on the test data.

## Section 1: Implementing `MyKNNClassifier`

In the cell below, we use the "magic command" `%%writefile` to directly save the cell contents into a file named `knn.py`. This file contains the exact class requested in the prompt, with all `fit` and `predict` methods implemented.

**Key Implementation Notes:**
-   `fit(X, y)`: Simply stores the training data within the object.
-   `predict(X)`: Calls a helper function `_predict_one` for each sample in `X` (test set).
-   `_predict_one(x)`: This method is the heart of the algorithm:
    1.  Calculates the distance from sample `x` to *all* training samples (`self.X_train`) based on the metric (Euclidean or Manhattan). (Fully vectorized with NumPy)
    2.  Finds the `k` nearest neighbors using `np.argsort`.
    3.  Performs voting based on `weighted=True/False`.

In [8]:
%%writefile knn.py
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from collections import Counter

class MyKNNClassifier(BaseEstimator, ClassifierMixin):
    """
    K-Nearest Neighbors Algorithm implementation using NumPy.
    """
    def __init__(self, n_neighbors=3, metric="euclidean", weighted=False):
        self.n_neighbors = n_neighbors
        self.metric = metric
        self.weighted = weighted
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):
        """
        Stores the training data.
        """
        # Store X and y in self.X_train and self.y_train.
        self.X_train = X
        self.y_train = y
        # Return self for sklearn compatibility
        return self

    def _calculate_distances(self, x):
        """ 
        Calculates the distance from sample x to all training data.
        """
        if self.metric == "euclidean":
            # Euclidean distance: sqrt(sum((x_i - y_i)^2))
            return np.sqrt(np.sum((self.X_train - x)**2, axis=1))
        elif self.metric == "manhattan":
            # Manhattan distance: sum(|x_i - y_i|)
            return np.sum(np.abs(self.X_train - x), axis=1)
        else:
            raise ValueError(f"Unknown metric: {self.metric}")

    def _predict_one(self, x):
        """
        Predicts the label for a single sample x.
        """
        # 1. Calculate distances
        distances = self._calculate_distances(x)
        
        # 2. Find the k nearest neighbors (indices and distances)
        k_indices = np.argsort(distances)[:self.n_neighbors]
        k_nearest_labels = self.y_train[k_indices]
        k_nearest_distances = distances[k_indices]

        # 3. Perform voting
        if not self.weighted:
            # Unweighted mode: Simple voting (finding the most frequent)
            # Use Counter to find the simplest most frequent item
            most_common = Counter(k_nearest_labels).most_common(1)
            return most_common[0][0]
        else:
            # Weighted mode: Weighted voting (based on 1 / distance)
            # Add a very small value (epsilon) to prevent division by zero
            epsilon = 1e-10
            weights = 1 / (k_nearest_distances + epsilon)
            
            # Sum of weights for each class
            class_scores = {}
            for label, weight in zip(k_nearest_labels, weights):
                if label not in class_scores:
                    class_scores[label] = 0
                class_scores[label] += weight
            
            # Return the class with the highest score (sum of weights)
            return max(class_scores, key=class_scores.get)

    def predict(self, X):
        """
        Predicts labels for a set of samples X.
        """
        # TODO:
        # 1. Call the _predict_one method for each sample in X.
        # 2. Return the results as a NumPy array.
        if self.X_train is None or self.y_train is None:
            raise RuntimeError("Model has not been fit() yet.")
            
        y_pred = [self._predict_one(x) for x in X]
        return np.array(y_pred)


Overwriting knn.py


## Section 2: Loading and Preparing the Data

Now that our class in `knn.py` is ready, we `import` the necessary libraries and the class itself. Then, we load the Iris data, scale it, and split it into training and test sets.

In [2]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score

# Import the class we wrote
try:
    from knn import MyKNNClassifier
    print("MyKNNClassifier class successfully imported from knn.py.")
except ImportError:
    print("Error: knn.py file not found. Please run the previous cell.")

MyKNNClassifier class successfully imported from knn.py.


In [3]:
# 1. Load the data
iris = load_iris()
X = iris.data
y = iris.target
target_names = iris.target_names

print(f"Data shape (X): {X.shape}")
print(f"Label shape (y): {y.shape}")

Data shape (X): (150, 4)
Label shape (y): (150,)


### Preprocessing: Scaling and Splitting

The KNN algorithm is distance-based, so if features have different scales (e.g., one from 0 to 1 and another from 0 to 1000), the feature with the larger scale will dominate the distance calculations. 

**1. Scaling:** We use `StandardScaler` to ensure all features have a mean of 0 and a variance of 1.
**2. Splitting:** We divide the data into training (75%) and test (25%) sets. We use `stratify=y` to ensure the class ratios remain the same in both sections.

In [4]:
# 1. Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
# 1. Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Number of training samples: {len(X_train_scaled)}")
print(f"Number of test samples: {len(X_test_scaled)}")

Number of training samples: 112
Number of test samples: 38


## Section 3: Hyperparameter Tuning with Cross-Validation

To find the best combination of `n_neighbors`, `metric`, and `weighted`, we use `GridSearchCV`. This tool automatically tests all possible combinations using Cross-Validation (here, we define 5-Fold CV) on the **training data**.

In [5]:
# Define the parameter space for the search
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 13],    # Different values for K
    'metric': ['euclidean', 'manhattan'],  # The two implemented metrics
    'weighted': [True, False]               # Both weighted and unweighted modes
}

# Define the Cross-Validation strategy (5-Folds)
cv_strategy = KFold(n_splits=5, shuffle=True, random_state=42)

# Create the GridSearchCV tool using our custom model
grid_search = GridSearchCV(
    estimator=MyKNNClassifier(), 
    param_grid=param_grid,
    cv=cv_strategy,
    scoring='accuracy',  # Our evaluation metric is Accuracy
    n_jobs=-1            # Use all CPU cores for faster processing
)

print("Running Grid Search to find the best hyperparameters...")
# Run the search only on the training data
grid_search.fit(X_train_scaled, y_train)

print("Search completed.")

Running Grid Search to find the best hyperparameters...
Search completed.


In [6]:
# Display search results
print(f"Best score (Accuracy) in Cross-Validation: {grid_search.best_score_:.4f}")
print(f"Best configuration (parameters) found: {grid_search.best_params_}")

Best score (Accuracy) in Cross-Validation: 0.9735
Best configuration (parameters) found: {'metric': 'euclidean', 'n_neighbors': 11, 'weighted': True}


## Section 4: Final Evaluation and Reporting Results

Now that `GridSearchCV` has found the best model (with the best parameters) and stored it in `grid_search.best_estimator_`, we use this final model to make predictions on the **test data** (`X_test`). This data has not been seen during the training and optimization process, making our final evaluation valid.

We report the results using `classification_report`.

In [7]:
# Extract the best model found and trained by GridSearch
best_knn_model = grid_search.best_estimator_

# Predict on the test set
y_pred = best_knn_model.predict(X_test_scaled)

# Calculate and print the final report
print("--- Final Classification Report on the Test Set ---")
print(f"Model Configuration: {grid_search.best_params_}\n")

print(classification_report(y_test, y_pred, target_names=target_names))

--- Final Classification Report on the Test Set ---
Model Configuration: {'metric': 'euclidean', 'n_neighbors': 11, 'weighted': True}

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        12
  versicolor       0.93      1.00      0.96        13
   virginica       1.00      0.92      0.96        13

    accuracy                           0.97        38
   macro avg       0.98      0.97      0.97        38
weighted avg       0.98      0.97      0.97        38



### Final Analysis

The report above shows the final performance of our optimized model on unseen data.

-   **Precision:** Out of all samples the model predicted as a specific class, what percentage were actually correct?
-   **Recall:** Out of all actual samples of a specific class, what fraction did the model correctly identify?
-   **F1-Score:** The harmonic mean of Precision and Recall.
-   **Accuracy:** The percentage of total correct predictions.

As observed, our `MyKNNClassifier` implementation using NumPy achieved very good results (usually over 95% accuracy) on the Iris dataset, and the parameter optimization process was performed correctly.